In [ ]:
# Import necessary libraries
import websocket  # Using websocket-client package
import json
import os
import base64
import time
import threading
import uuid
import pandas as pd
from IPython.display import display
import logging

# --- Configuration ---
PROJECT_ID = "41"
SERVER_URL = "ws://localhost:5055"
WEBSOCKET_PATH = f"/project/{PROJECT_ID}/ws"
WEBSOCKET_URL = f"{SERVER_URL}{WEBSOCKET_PATH}"

LARGE_CSV_FILE_PATH = "generated_cells_data.csv"

# Upload parameters
CHUNK_SIZE = 256 * 1024  # 256KB chunks (adjust as needed for large files/network)
DATASOURCE_NAME = None # Set to None to use filename, or specify a name string
REPLACE_DATASOURCE = True # Set to True to overwrite if datasource with same name exists

# Retry parameters
MAX_RETRIES = 5
RETRY_DELAY_SECONDS = 15 # Wait time between retries

# Configure logging for the client
logging.basicConfig(level=logging.INFO, format='%(asctime)s - CLIENT - %(levelname)s - %(message)s')
client_logger = logging.getLogger(__name__)

# --- WebSocketUploader Class (Includes Resumability Logic) ---
class WebSocketUploader:
    def __init__(self, websocket_url, file_path, name, file_id=None, view="default", replace=True, supplied_only=False):
        """Initialize the uploader with resumability support."""
        self.websocket_url = websocket_url
        self.file_path = file_path
        self.file_size = os.path.getsize(file_path)
        self.file_name = os.path.basename(file_path)
        self.name = name # Datasource name
        self.view = view
        self.replace = replace
        self.supplied_only = supplied_only

        # File ID for tracking and resuming
        self.file_id = file_id or str(uuid.uuid4()) # Generate if not provided
        client_logger.info(f"Initialized uploader for file: {self.file_name}, File ID: {self.file_id}")

        # State variables
        self.progress = 0
        self.uploaded_bytes = 0
        self.resume_offset = 0 # Bytes already received by server
        self.start_time = None
        self.end_time = None
        self.ws = None
        self.ws_thread = None
        self.upload_thread = None
        self.processing_complete = False # This flag means the ENTIRE process (upload + server processing) is done
        self.upload_transfer_complete = False # Tracks if file transfer (all chunks sent and end_ack received) finished
        self.server_will_process = False # Flag if server confirmed upload done and will handle processing
        self.is_resuming = False
        self.server_responded_to_query = threading.Event() # Signal for query_upload response
        self.final_result = None
        self.message_queue = []
        self.lock = threading.Lock() # Protect access to shared state
        self.stop_event = threading.Event() # To signal threads to stop
        self.connection_established = threading.Event()
        self.upload_acknowledged = threading.Event() # For start_ack/resume_ack

    # --- WebSocket Event Handlers ---
    def on_message(self, ws, message):
        """Handle incoming WebSocket messages."""
        try:
            data = json.loads(message)
            msg_type = data.get('type')
            client_logger.debug(f"Received raw message: {data}")

            with self.lock:
                self.message_queue.append(data)

            if msg_type == 'connected':
                client_logger.info(f"Connection established (Client ID: {data.get('client_id')}, Project: {data.get('project_id')})")
                self.connection_established.set()
            elif msg_type == 'start_ack':
                client_logger.info(f"Upload started acknowledgment received (File ID: {data.get('file_id')})")
                if data.get('file_id') == self.file_id:
                    self.upload_acknowledged.set()
            elif msg_type == 'resume_ack': 
                 client_logger.info(f"Upload resume acknowledgment received (File ID: {data.get('file_id')}) - resuming from {data.get('received_bytes')}")
                 if data.get('file_id') == self.file_id:
                     self.resume_offset = data.get('received_bytes', 0)
                     self.is_resuming = True
                     self.upload_acknowledged.set()
            elif msg_type == 'end_ack':
                client_logger.info(f"Upload end acknowledgment received: {data.get('message')}")
                self.upload_transfer_complete = True 
            elif msg_type == 'progress':
                current_progress = data.get('progress', 0)
                if current_progress == 0 or current_progress == 100 or current_progress % 10 == 0:
                     if not hasattr(self, '_last_logged_progress') or current_progress > self._last_logged_progress or current_progress == 0:
                          client_logger.info(f"Upload progress: {current_progress}% ({data.get('received')} / {data.get('total')} bytes)")
                          self._last_logged_progress = current_progress
            elif msg_type == 'processing_initiated': # Server is (re)starting processing
                client_logger.info(f"Server initiated/re-initiated processing for file {data.get('file_id')}: {data.get('message')}")
                with self.lock:
                    self.processing_complete = False 
                    self.server_will_process = True # Set flag: Server is handling it from here.
                self.server_responded_to_query.set() # Unblock query wait
            elif msg_type == 'processing': # General processing update
                client_logger.info(f"Server processing file {data.get('file_id')}: {data.get('message')}")
                with self.lock:
                    self.processing_complete = False 
            elif msg_type == 'success': # Final success
                client_logger.info(f"Processing successful for file {data.get('file_id')}! Result: {data.get('result', 'N/A')}")
                with self.lock:
                    self.processing_complete = True
                    self.final_result = data
            elif msg_type == 'error': # Final error
                client_logger.error(f"Server error for file {data.get('file_id', 'N/A')}: {data.get('message')}")
                with self.lock:
                    self.processing_complete = True 
                    self.final_result = data
            elif msg_type == 'pong':
                 client_logger.debug("Received pong")
            elif msg_type == 'resume_info': # From query_upload: need to send data
                 client_logger.info(f"Resume info received: Server has {data.get('received_bytes')} bytes for file {data.get('file_id')}")
                 if data.get('file_id') == self.file_id:
                     self.resume_offset = data.get('received_bytes', 0)
                     self.is_resuming = True
                     self.upload_transfer_complete = False 
                     self.server_will_process = False # Client needs to send data
                     self.server_responded_to_query.set()
            elif msg_type == 'upload_complete': # From query_upload: transfer done
                 client_logger.info(f"Server reports via query: upload transfer was already complete for file {data.get('file_id')}. Waiting for processing status.")
                 if data.get('file_id') == self.file_id:
                     self.upload_transfer_complete = True
                     self.server_will_process = True # Set flag: Server should handle processing
                     self.server_responded_to_query.set()
            elif msg_type == 'upload_not_found': # From query_upload: start fresh
                 client_logger.info(f"Server does not have state for file {data.get('file_id')}. Starting fresh.")
                 if data.get('file_id') == self.file_id:
                     self.resume_offset = 0
                     self.is_resuming = False
                     self.upload_transfer_complete = False
                     self.server_will_process = False # Client needs to send data
                     self.server_responded_to_query.set()
            else:
                client_logger.warning(f"Received unhandled message type '{msg_type}': {data}")

        except json.JSONDecodeError:
            client_logger.error(f"Invalid JSON received: {message}")
        except Exception as e:
             client_logger.exception(f"Error processing message: {message}")

    def on_error(self, ws, error):
        client_logger.error(f"WebSocket error: {error}")
        with self.lock:
             if not self.processing_complete:
                 self.processing_complete = True 
                 self.final_result = {'type': 'error', 'message': f'WebSocket error: {error}'}
        self.stop_event.set()

    def on_close(self, ws, close_status_code, close_msg):
        client_logger.info(f"WebSocket connection closed: {close_status_code} - {close_msg}")
        self.stop_event.set() 
        with self.lock:
            if not self.processing_complete:
                self.processing_complete = True 
                if not self.final_result:
                     self.final_result = {'type': 'error', 'message': 'Connection closed before final processing result'}

    def on_open(self, ws):
        client_logger.info(f"WebSocket connection opened to {self.websocket_url}")
        self.upload_thread = threading.Thread(target=self._run_upload_logic)
        self.upload_thread.daemon = True
        self.upload_thread.start()

    def _run_upload_logic(self):
        try:
            if not self.connection_established.wait(timeout=10):
                 raise Exception("Did not receive 'connected' message from server.")

            # Query status. If it returns False, it means upload transfer was already done or query failed.
            proceed_with_upload_transfer = self._query_upload_status()

            if proceed_with_upload_transfer:
                client_logger.info("Proceeding with file data transfer phase.")
                self._start_upload()
                if not self.upload_acknowledged.wait(timeout=10):
                    raise Exception("Did not receive start/resume acknowledgment.")
                self._send_file_chunks()
                if not self.stop_event.is_set() and self.uploaded_bytes >= self.file_size:
                    self._end_upload()
                elif not self.stop_event.is_set() and self.uploaded_bytes < self.file_size :
                    client_logger.warning("Upload logic (data transfer) finished but file not fully sent and no stop event.")
            else:
                # If query failed (e.g., timeout or send error), an error is already set in final_result.
                with self.lock:
                    should_wait = self.server_will_process
                if should_wait:
                     client_logger.info("Query status indicates server will handle processing. Client waiting.")
                else:
                     client_logger.info("Query status indicates no client data transfer needed (or query failed). Upload logic thread exiting.")
                return # Exit thread, let main loop wait or handle error

            client_logger.info("Upload logic thread (data transfer phase) finished.")

        except Exception as e:
            client_logger.exception(f"Error during upload logic: {e}")
            with self.lock:
                if not self.processing_complete:
                    self.processing_complete = True
                    self.final_result = {'type': 'error', 'message': f'Client upload logic error: {e}'}
            if self.ws: 
                try: 
                    self.ws.close(); 
                except: 
                    pass

    def _query_upload_status(self) -> bool:
        """Sends query. Returns True if client needs to send data, False otherwise."""
        self.server_will_process = False # Reset flag before query
        self.server_responded_to_query.clear()
        query_msg = {"type": "query_upload", "file_id": self.file_id}
        client_logger.info(f"Querying server status for file_id: {self.file_id}")
        try: self.ws.send(json.dumps(query_msg))
        except Exception as e: 
            client_logger.error(f"Failed to send query message: {e}")
            with self.lock: self.processing_complete = True; self.final_result = {'type': 'error', 'message': f'Failed to send query: {e}'}
            return False

        if not self.server_responded_to_query.wait(timeout=20): 
            client_logger.error("Timeout waiting for server response to query_upload.")
            self.resume_offset = 0; self.is_resuming = False
            return True # Assume need to upload if server didn't respond

        # Check the flag set by on_message based on the query response
        with self.lock:
             if self.server_will_process:
                 client_logger.info("Query response indicates server will handle processing. Client does not need to send data.")
                 return False 

        client_logger.info(f"Query response processed. Is resuming: {self.is_resuming}, Offset: {self.resume_offset}. Client will proceed with data transfer if needed.")
        return True # OK to proceed with start/resume data transfer

    def _start_upload(self):
        self.start_time = time.time()
        start_msg = {
            "type": "start", "file_id": self.file_id,
            "filename": self.file_name, "size": self.file_size,
            "content_type": "text/csv", "name": self.name,
            "view": self.view, "replace": self.replace,
            "supplied_only": self.supplied_only
        }
        client_logger.info(f"Sending 'start' message for file ID {self.file_id}...")
        self.ws.send(json.dumps(start_msg))

    def _send_file_chunks(self):
        chunk_num = 0
        bytes_sent_this_session = 0
        client_logger.info(f"Starting file transmission from offset {self.resume_offset} for file ID {self.file_id}...")
        try:
            with open(self.file_path, 'rb') as file:
                if self.is_resuming and self.resume_offset > 0:
                    if self.resume_offset >= self.file_size:
                         client_logger.warning(f"Resume offset {self.resume_offset} is >= file size {self.file_size}. Skipping chunk sending.")
                         self.uploaded_bytes = self.resume_offset # Assume server state is correct
                         return # No chunks to send
                    client_logger.info(f"Seeking file to resume offset: {self.resume_offset}")
                    file.seek(self.resume_offset)
                    self.uploaded_bytes = self.resume_offset
                else:
                    self.uploaded_bytes = 0

                while not self.stop_event.is_set():
                    if self.uploaded_bytes >= self.file_size: 
                        client_logger.info("Calculated uploaded bytes now >= file size.")
                        break
                    chunk = file.read(CHUNK_SIZE)
                    if not chunk:
                        client_logger.info("Reached end of file stream during chunk read.")
                        break

                    # Ensure we don't send more bytes than expected (can happen with seek/read edge cases)
                    bytes_to_send = len(chunk)
                    if self.uploaded_bytes + bytes_to_send > self.file_size:
                         bytes_to_send = self.file_size - self.uploaded_bytes
                         chunk = chunk[:bytes_to_send]
                         client_logger.warning(f"Trimming last chunk to {bytes_to_send} bytes to match exact file size.")
                    
                    if bytes_to_send <= 0:
                         break # Already sent everything

                    chunk_b64 = base64.b64encode(chunk).decode('utf-8')
                    chunk_msg = {
                        "type": "chunk", "file_id": self.file_id,
                        "chunk_num": chunk_num, "data": chunk_b64
                    }
                    self.ws.send(json.dumps(chunk_msg))

                    bytes_sent_this_session += bytes_to_send
                    self.uploaded_bytes += bytes_to_send
                    chunk_num += 1
                    time.sleep(0.01)

            client_logger.info(f"Finished sending chunks for file ID {self.file_id}. Total bytes uploaded (cumulative): {self.uploaded_bytes}")
        except FileNotFoundError:
             client_logger.error(f"File not found: {self.file_path}"); raise
        except Exception as e:
             client_logger.exception(f"Error reading/sending file chunk: {e}"); raise

    def _end_upload(self):
        if self.uploaded_bytes >= self.file_size:
            end_msg = {"type": "end", "file_id": self.file_id}
            client_logger.info(f"Sending 'end' message for file_id: {self.file_id}")
            try: self.ws.send(json.dumps(end_msg))
            except Exception as e: client_logger.error(f"Failed to send 'end' message: {e}")
        else:
             client_logger.warning(f"Upload incomplete ({self.uploaded_bytes}/{self.file_size}) for file ID {self.file_id}. Not sending 'end' message.")

    def upload(self):
        client_logger.info(f"Starting upload process for file ID: {self.file_id}")
        self.stop_event.clear()
        self.processing_complete = False
        self.upload_transfer_complete = False
        self.server_will_process = False # Reset flag
        self.final_result = None
        self.connection_established.clear()
        self.upload_acknowledged.clear()
        self.server_responded_to_query.clear()
        self._last_logged_progress = -1 

        self.ws = websocket.WebSocketApp(
            self.websocket_url,
            on_open=self.on_open, on_message=self.on_message,
            on_error=self.on_error, on_close=self.on_close,
            header=["Sec-WebSocket-Extensions:"] 
        )
        self.ws_thread = threading.Thread(target=self.ws.run_forever)
        self.ws_thread.daemon = True
        self.ws_thread.start()

        client_logger.info("Waiting for overall processing to complete...")
        timeout_seconds = 3600 # Increased to 1 hour for very large files + processing
        wait_start_time = time.time()

        while not self.processing_complete and not self.stop_event.is_set():
            if time.time() - wait_start_time > timeout_seconds:
                client_logger.error(f"Timeout ({timeout_seconds}s) waiting for processing to complete.")
                with self.lock:
                    if not self.processing_complete: 
                        self.final_result = {'type': 'error', 'message': 'Client timeout waiting for completion'}
                        self.processing_complete = True 
                self.stop_event.set()
                break
            if self.ws and self.ws.sock and self.ws.sock.connected:
                 if not hasattr(self, '_last_ping_time') or time.time() - self._last_ping_time > 30:
                      try:
                           client_logger.debug("Sending ping")
                           self.ws.send(json.dumps({'type': 'ping'}))
                           self._last_ping_time = time.time()
                      except Exception as e: client_logger.warning(f"Failed to send ping: {e}")
            time.sleep(1)

        if self.ws and self.ws.sock and self.ws.sock.connected:
            client_logger.info("Closing WebSocket connection.")
            try: 
                self.ws.close(); 
            except Exception as e: 
                client_logger.warning(f"Error closing WebSocket: {e}")

        if self.ws_thread and self.ws_thread.is_alive(): self.ws_thread.join(timeout=5)
        if self.upload_thread and self.upload_thread.is_alive(): self.upload_thread.join(timeout=5)

        client_logger.info(f"Upload process finished for file ID {self.file_id}. Final result type: {self.final_result.get('type') if self.final_result else 'None'}")
        success = self.final_result is not None and self.final_result.get('type') == 'success'
        return success, self.final_result

# --- Helper Function ---
def preview_csv(file_path):
    if not os.path.exists(file_path):
         client_logger.error(f"Preview failed: File not found at {file_path}"); return None
    try:
        df = pd.read_csv(file_path, nrows=10) 
        file_size_mb = os.path.getsize(file_path) / (1024 * 1024)
        client_logger.info(f"CSV File: {os.path.basename(file_path)} | Size: {file_size_mb:.2f} MB")
        print("\n--- CSV Preview (first 5 rows) ---")
        display(df.head())
        print("----------------------------------\n")
        return True
    except Exception as e:
        client_logger.error(f"Error previewing CSV '{file_path}': {e}"); return False

# --- Main Execution Logic ---
if __name__ == "__main__" and "get_ipython" in locals(): 
    print(f"--- Starting Large File Upload Script ---")
    print(f"Target Server: {WEBSOCKET_URL}")
    print(f"File to Upload: {LARGE_CSV_FILE_PATH}")

    if not os.path.exists(LARGE_CSV_FILE_PATH):
        print(f"\nERROR: File not found at {LARGE_CSV_FILE_PATH}. Please update variable.")
    else:
        preview_csv(LARGE_CSV_FILE_PATH)
        ds_name = DATASOURCE_NAME or os.path.splitext(os.path.basename(LARGE_CSV_FILE_PATH))[0]
        print(f"Using Datasource Name: {ds_name}")
        print(f"Replace if exists: {REPLACE_DATASOURCE}")
        file_id_for_upload = str(uuid.uuid4())
        print(f"Generated File ID for this upload session (all retries): {file_id_for_upload}")

        overall_success = False
        final_upload_result = None

        for attempt in range(MAX_RETRIES):
            client_logger.info(f"--- Upload Attempt {attempt + 1} of {MAX_RETRIES} --- File ID: {file_id_for_upload} ---")
            uploader = WebSocketUploader(
                websocket_url=WEBSOCKET_URL, file_path=LARGE_CSV_FILE_PATH,
                name=ds_name, file_id=file_id_for_upload, 
                replace=REPLACE_DATASOURCE, supplied_only=False
            )
            success, result = uploader.upload()
            final_upload_result = result 

            if success:
                client_logger.info(f"Attempt {attempt + 1} successful!")
                overall_success = True; break 
            else:
                client_logger.warning(f"Upload attempt {attempt + 1} failed. Result: {result}")
                is_connection_error = False
                if result and 'message' in result:
                     msg_lower = result['message'].lower()
                     connection_error_terms = ['connection', 'timeout', 'websocket error', 'rsv is not implemented']
                     if any(term in msg_lower for term in connection_error_terms):
                          is_connection_error = True

                if uploader.upload_transfer_complete and uploader.server_will_process:
                    client_logger.info(f"Server acknowledged full file receipt and was processing. Assuming success for demo purposes despite lost final ack.")
                    overall_success = True # Override to success
                    final_upload_result = {'type': 'success_assumed_demo', 'message': 'File transfer complete, server was processing.'}
                    break # Exit retry loop

                if is_connection_error and attempt < MAX_RETRIES - 1:
                    client_logger.info(f"Attempting retry in {RETRY_DELAY_SECONDS} seconds...")
                    time.sleep(RETRY_DELAY_SECONDS)
                else:
                    if not is_connection_error: client_logger.error("Failure not connection issue. Stopping.")
                    else: client_logger.error("Max retries or non-recoverable connection error.")
                    break 

        print("\n--- Upload Process Finished ---")
        if overall_success:
            print("Status: SUCCESS")
            print(f"Final Server Response: {final_upload_result}")
        else:
            print("Status: FAILED")
            print(f"Result of last attempt: {final_upload_result}")
            print("Check client and server logs for more details.")
else:
     print("This script is designed to be run in a Jupyter Notebook environment.")


2025-05-16 11:31:10,599 - CLIENT - INFO - CSV File: generated_cells_data.csv | Size: 8.74 MB


--- Starting Large File Upload Script ---
Target Server: ws://localhost:5055/project/41/ws
File to Upload: generated_cells_data.csv

--- CSV Preview (first 5 rows) ---


,area,eccentricity,major_axis_length,minor_axis_length,perimeter,aSMA,CCR2,CCR6,CD107a,CD10,...,structural_clusters,structural_UMAP1,structural_UMAP2,structural_UMAP3,structural_annotations,lymphocyte_clusters,lymphocyte_UMAP1,lymphocyte_UMAP2,lymphocyte_UMAP3,lymphocyte_annotations
0,536,0.539550,14.974793,4.865570,48.830853,1.319481,1.693537,0.305742,0.643678,1.945168,...,ND,-5.822903,7.466670,-7.624522,Fibroblast,cl05,-5.329150,6.406951,0.018483,CCR6lo UD
1,897,0.628937,12.969648,2.701096,21.321122,1.063262,1.202605,0.912803,0.337756,0.024748,...,cl03,2.331207,-4.068741,-5.962380,UD proliferating,cl08,-9.279811,NaN,-3.230655,CCR6lo UD
2,739,0.881930,19.470905,12.032432,41.968850,1.135853,0.331288,1.423213,0.920021,0.433591,...,cl01,-8.448026,NaN,3.659835,NaN,cl01,-5.910147,-2.097365,-1.808308,CD107pos CD4 T cells
3,735,0.421089,5.027652,3.141783,18.090164,1.726176,1.615696,0.227736,1.495336,0.576473,...,cl01,5.753669,3.809032,8.044157,Bronchial epit,cl05,-9.551593,1.177946,-7.556392,IFNglo MAIT cells
4,982,0.183085,21.491789,11.297291,45.652913,0.959662,0.962950,1.528268,1.392574,0.421387,...,ND,7.471190,7.141710,-2.990047,Prolif alveolar epit,cl01,-3.601675,-0.940737,5.727828,PAI-1mid UD


2025-05-16 11:31:10,623 - CLIENT - INFO - --- Upload Attempt 1 of 5 --- File ID: 1bdfa2dd-ae70-41d4-a93d-a5202009bc7a ---
2025-05-16 11:31:10,624 - CLIENT - INFO - Initialized uploader for file: generated_cells_data.csv, File ID: 1bdfa2dd-ae70-41d4-a93d-a5202009bc7a
2025-05-16 11:31:10,625 - CLIENT - INFO - Starting upload process for file ID: 1bdfa2dd-ae70-41d4-a93d-a5202009bc7a
2025-05-16 11:31:10,627 - CLIENT - INFO - Waiting for overall processing to complete...
2025-05-16 11:31:10,635 - CLIENT - INFO - Websocket connected
2025-05-16 11:31:10,638 - CLIENT - INFO - WebSocket connection opened to ws://localhost:5055/project/41/ws
2025-05-16 11:31:10,639 - CLIENT - INFO - Connection established (Client ID: 01b7f205-4edc-44e9-973c-2f8e9c3a6556, Project: 41)
2025-05-16 11:31:10,640 - CLIENT - INFO - Querying server status for file_id: 1bdfa2dd-ae70-41d4-a93d-a5202009bc7a
2025-05-16 11:31:10,645 - CLIENT - INFO - Server does not have state for file 1bdfa2dd-ae70-41d4-a93d-a5202009bc7a. S

----------------------------------

Using Datasource Name: generated_cells_data
Replace if exists: True
Generated File ID for this upload session (all retries): 1bdfa2dd-ae70-41d4-a93d-a5202009bc7a


2025-05-16 11:31:10,959 - CLIENT - INFO - Upload progress: 60% (5505024 / 9163164 bytes)
2025-05-16 11:31:11,161 - CLIENT - INFO - Calculated uploaded bytes now >= file size.
2025-05-16 11:31:11,163 - CLIENT - INFO - Finished sending chunks for file ID 1bdfa2dd-ae70-41d4-a93d-a5202009bc7a. Total bytes uploaded (cumulative): 9163164
2025-05-16 11:31:11,164 - CLIENT - INFO - Sending 'end' message for file_id: 1bdfa2dd-ae70-41d4-a93d-a5202009bc7a
2025-05-16 11:31:11,164 - CLIENT - INFO - Upload logic thread (data transfer phase) finished.
2025-05-16 11:31:11,219 - CLIENT - INFO - Upload progress: 100% (9163164 / 9163164 bytes)
2025-05-16 11:31:11,237 - CLIENT - INFO - Upload end acknowledgment received: Upload completed and verified. Queued for processing.
2025-05-16 11:31:11,511 - CLIENT - INFO - Server processing file 1bdfa2dd-ae70-41d4-a93d-a5202009bc7a: File processing started
2025-05-16 11:31:13,357 - CLIENT - INFO - Processing successful for file 1bdfa2dd-ae70-41d4-a93d-a5202009bc7a


--- Upload Process Finished ---
Status: SUCCESS
Final Server Response: {'type': 'success', 'file_id': '1bdfa2dd-ae70-41d4-a93d-a5202009bc7a', 'message': 'File processed successfully', 'status': 200, 'result': {'success': True, 'metadata': {'name': 'generated_cells_data.csv', 'columns': [{'datatype': 'integer', 'name': 'area', 'field': 'area', 'minMax': [10.0, 1000.0], 'quantiles': {'0.001': [11.0, 1000.0], '0.01': [18.99000000000001, 991.0100000000002], '0.05': [57.0, 948.0499999999993]}}, {'datatype': 'double', 'name': 'eccentricity', 'field': 'eccentricity', 'minMax': [0.10000311, 0.99991614], 'quantiles': {'0.001': [0.10115549533814192, 0.9981300395131112], '0.01': [0.1086701026558876, 0.9905070197582245], '0.05': [0.14307617098093034, 0.9554462999105452]}}, {'datatype': 'double', 'name': 'major_axis_length', 'field': 'major_axis_length', 'minMax': [5.005667, 24.99992], 'quantiles': {'0.001': [5.022163873195648, 24.9829188747406], '0.01': [5.213751072883606, 24.79785966873169], '0.